# Data prepocessing
1. Use mediapipe to draw eye region shape with only black and white color.
2. Use mediapipe to draw trimap, which means white, gray, and black color to distinguish foreground, blurred contour part and background, based on previous image to position the make-up region.
3. Use Pymatting with trimap and original image to find make-up region.
4. As result, step 1 figure will be independent variable x and step 3 figure will be dependent variable y.

## Step 1 and 2

In [11]:
import cv2
import mediapipe as mp
import numpy as np
import os
# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)
# Initialize drawing utilities
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Load image using OpenCV
silhouette =  [
    10,  338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288,
    397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136,
    172, 58,  132, 93,  234, 127, 162, 21,  54,  103, 67,  109
  ]


rightEyeUpper0 =  [246, 161, 160, 159, 158, 157, 173]
rightEyeUpper1 = [247, 30, 29, 27, 28, 56, 190]
rightEyeLower0 = [33, 7, 163, 144, 145, 153, 154, 155, 133]
rightEyeLower1 = [130, 25, 110, 24, 23, 22, 26, 112, 243]
rightEyeLower3 = [143, 111, 117, 118, 119, 120, 121, 128, 245]
rightEyebrowLower = [ 124, 46, 53, 52, 65, 193]

leftEyeUpper0 = [466, 388, 387, 386, 385, 384, 398]
leftEyeUpper1 = [467, 260, 259, 257, 258, 286, 414]
leftEyeLower0 = [263, 249, 390, 373, 374, 380, 381, 382, 362]
leftEyeLower1 = [359, 255, 339, 254, 253, 252, 256, 341, 463]
leftEyeLower3 = [372, 340, 346, 347, 348, 349, 350, 357, 465]

rightEyeLower0.reverse()
rightEyeLower1.reverse()
rightEyeLower3.reverse()
leftEyeLower0.reverse()
leftEyeLower1.reverse()
leftEyeLower3.reverse()

leftEyebrowLower = [276, 283, 282, 295, 285]



# Replace with your image path
# Defien all original settings

def eye_region_generator(image,index, region_type: str, image_output_path:str):
    
    righteyeout_position = []
    righteyein_position = []
    lefteyeout_position = []
    lefteyein_position = []
    righteyemargin_position = []
    lefteyemargin_position = []
    # Convert the BGR image to RGB
    height, width, _ = image.shape
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image to find facial landmarks
    results = face_mesh.process(rgb_image)

    # Check if landmarks were detected
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Draw facial landmarks on the image
            mp_drawing.draw_landmarks(
                image=image,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())

            # Right Eye out region
            for i in rightEyebrowLower+rightEyeLower3:
                lm = face_landmarks.landmark[i]
                x, y = int(lm.x * width), int(lm.y * height)
                righteyeout_position.append([x,y])
            righteyeout_position = np.array(righteyeout_position, dtype=np.int32).reshape((-1, 1, 2))
            
            # Right Eye margin region        
            for i in rightEyeUpper1+rightEyeLower1:
                lm = face_landmarks.landmark[i]
                x, y = int(lm.x * width), int(lm.y * height)
                righteyemargin_position.append([x,y])
            righteyemargin_position = np.array(righteyemargin_position, dtype=np.int32).reshape((-1, 1, 2))
            cv2.fillPoly(image, [righteyemargin_position],(255,255,255))  
            
            # Right Eye in region
            for i in rightEyeLower0+rightEyeUpper0:
                lm = face_landmarks.landmark[i]
                x, y = int(lm.x * width), int(lm.y * height)
                righteyein_position.append([x,y])
            righteyein_position = np.array(righteyein_position, dtype=np.int32).reshape((-1, 1, 2))        
            cv2.fillPoly(image, [righteyein_position],(0, 0, 0))      
                    
            # Left Eye out region
            for i in leftEyebrowLower+leftEyeLower3:
                lm = face_landmarks.landmark[i]
                x, y = int(lm.x * width), int(lm.y * height)
                lefteyeout_position.append([x,y])
            lefteyeout_position = np.array(lefteyeout_position, dtype=np.int32).reshape((-1, 1, 2))
            
            # Left Eye margin region        
            for i in leftEyeUpper1+leftEyeLower1:
                lm = face_landmarks.landmark[i]
                x, y = int(lm.x * width), int(lm.y * height)
                lefteyemargin_position.append([x,y])
            lefteyemargin_position = np.array(lefteyemargin_position, dtype=np.int32).reshape((-1, 1, 2))
            cv2.fillPoly(image, [lefteyemargin_position],(255,255,255))        
            
            # Left Eye in region
            for i in leftEyeLower0+leftEyeUpper0:
                lm = face_landmarks.landmark[i]
                x, y = int(lm.x * width), int(lm.y * height)
                lefteyein_position.append([x,y])
            lefteyein_position = np.array(lefteyein_position, dtype=np.int32).reshape((-1, 1, 2))        
            cv2.fillPoly(image, [lefteyein_position],(0, 0, 0))       
        
            # Create a mask of the same size as the image, filled with zeros (black)
            
            mask = np.zeros_like(image)
            

            # # Fill the right eyebrow lower position on the mask with white color
            cv2.fillPoly(mask, [righteyeout_position], (255, 255, 255))
            cv2.fillPoly(mask, [lefteyeout_position], (255, 255, 255))
            # # Apply the inverted mask to the image
            image = cv2.bitwise_and(image, mask)        
                
            # Create mask2 to cover the image on color not black or white
            mask2 = None
            mask2 = np.zeros_like(image)
            # Create condition for non-black and non-white pixels
            condition = (image != [0, 0, 0])
            condition2 = (image == [255, 255, 255])
            condition = condition.all(axis=-1)
            condition2 = condition2.all(axis=-1)
            
            if region_type == 'allblackwhite':
                mask2[condition] = [255,255,255]
            elif region_type == 'trimap':
                mask2[condition] = [128,128,128]
            index = str(index)
            mask2[condition2] = [255,255,255]
            inverted_allblackwhite = cv2.bitwise_not(mask2)      
            image = cv2.bitwise_and(image, inverted_allblackwhite)        
            output_image_path = os.path.join(image_output_path, index)
            cv2.imwrite(output_image_path, mask2)



In [ ]:
image_path = '256img_lst' 
imagelist = os.listdir(image_path)

for index in imagelist:
    image = cv2.imread(image_path+'/'+ index)
    eye_region_generator(image,index,'allblackwhite','allblackwhite1')
    eye_region_generator(image,index,'trimap','trimap1')
    
    
    

face_mesh.close()
cv2.destroyAllWindows()

## Step 3: 
Use Pymatting. Choose to use rw algorithm.

In [13]:
import cv2
import mediapipe as mp
import numpy as np
import os
import pymatting as pym
 
imagepath = './256img_lst/'
trimappath = './trimap1'
pymtrwpath = './pymatting_outcome_rw1'
imagelist = os.listdir(imagepath)
trimaplist = os.listdir(trimappath)

for filename in imagelist:
    try:
        image = pym.load_image(imagepath +"/"+filename, "RGB")
        trimap = pym.load_image(trimappath+"/"+filename, "GRAY")

        alpha = pym.estimate_alpha_rw(
            image,
            trimap,
            laplacian_kwargs={},
            cg_kwargs={})
        pym.save_image(pymtrwpath +"/"+filename , alpha)
    except:
        print(f'{trimappath+"/"+filename} file has problem')

./trimap1/038.jpg file has problem
./trimap1/213.jpg file has problem
./trimap1/232.jpg file has problem
